In [183]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [184]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [185]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-04 19:00:00+00:00


In [186]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [187]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-04 13:03:38,521 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 13:03:38,530 INFO: Initializing external client
2025-03-04 13:03:38,531 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 13:03:39,174 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649


In [188]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.54s) 


In [189]:
df["pickup_hour"].max()

Timestamp('2025-03-04 19:00:00+0000', tz='Etc/UTC')

In [190]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,95,1.0,2025-03-03 03:00:00+00:00
1,179,1.0,2025-03-03 03:00:00+00:00
2,127,0.0,2025-03-03 03:00:00+00:00
3,90,71.0,2025-03-03 03:00:00+00:00
4,79,98.0,2025-03-03 03:00:00+00:00
...,...,...,...
5768,97,1.0,2025-03-04 19:00:00+00:00
5769,198,0.0,2025-03-04 19:00:00+00:00
5770,20,0.0,2025-03-04 19:00:00+00:00
5771,248,0.0,2025-03-04 19:00:00+00:00


In [191]:
#a.info()

In [192]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-04 13:03:41,926 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 13:03:41,926 INFO: Initializing external client
2025-03-04 13:03:41,926 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 13:03:42,561 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Current UTC time: 2025-03-04 18:03:41.926957+00:00
Next hour: 2025-03-04 19:00:00+00:00
Querying for date range: 2025-03-04 to 2025-03-05
Filtering for hour: 2025-03-04 19:00
Found 251 records


In [193]:
now = datetime.now(timezone.utc)

In [194]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
5522,30,0.0,2025-03-04 19:00:00+00:00
5523,199,0.0,2025-03-04 19:00:00+00:00
5524,31,0.0,2025-03-04 19:00:00+00:00
5525,239,203.0,2025-03-04 19:00:00+00:00
5526,167,1.0,2025-03-04 19:00:00+00:00
...,...,...,...
5768,97,1.0,2025-03-04 19:00:00+00:00
5769,198,0.0,2025-03-04 19:00:00+00:00
5770,20,0.0,2025-03-04 19:00:00+00:00
5771,248,0.0,2025-03-04 19:00:00+00:00


In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5773 entries, 0 to 5772
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  5773 non-null   int32                  
 1   predicted_demand    5773 non-null   float64                
 2   pickup_hour         5773 non-null   datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 112.9 KB


In [196]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [197]:
dt = current_date.ceil('h')


In [198]:
# fs = get_feature_store()  
# fg = fs.get_feature_group(  
#     name=config.FEATURE_GROUP_MODEL_PREDICTION,  
#     version=1  
# )  
# query = fg.select_all() 
# # query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
# query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
# results = query.read()

In [199]:
# fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

In [200]:
# results

In [201]:
# def fetch_next_hour_predictions():
#     # Get current UTC time and round up to next hour
#     now = datetime.now(timezone.utc)
#     next_hour = (now + timedelta(hours=2)).replace(minute=0, second=0, microsecond=0)

#     fs = get_feature_store()
#     fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

#     # Then filter for next hour in the DataFrame
#     df_hour = df[df["pickup_hour"] == next_hour]

#     print(f"Current UTC time: {now}")
#     print(f"Next hour: {next_hour}")
#     print(f"Found {len(df_hour)} records")
#     return df_hour

In [202]:
# predictions = fetch_next_hour_predictions()

In [203]:
# fs = get_feature_store()
# fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



In [204]:
# df = fg.read()

In [205]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,95,1.0,2025-03-03 03:00:00+00:00
1,179,1.0,2025-03-03 03:00:00+00:00
2,127,0.0,2025-03-03 03:00:00+00:00
3,90,71.0,2025-03-03 03:00:00+00:00
4,79,98.0,2025-03-03 03:00:00+00:00
...,...,...,...
5768,97,1.0,2025-03-04 19:00:00+00:00
5769,198,0.0,2025-03-04 19:00:00+00:00
5770,20,0.0,2025-03-04 19:00:00+00:00
5771,248,0.0,2025-03-04 19:00:00+00:00


In [206]:
df["pickup_hour"].max()

Timestamp('2025-03-04 19:00:00+0000', tz='Etc/UTC')

In [207]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-04 19:00:00+00:00


In [208]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
5522,30,0.0,2025-03-04 19:00:00+00:00
5523,199,0.0,2025-03-04 19:00:00+00:00
5524,31,0.0,2025-03-04 19:00:00+00:00
5525,239,203.0,2025-03-04 19:00:00+00:00
5526,167,1.0,2025-03-04 19:00:00+00:00
...,...,...,...
5768,97,1.0,2025-03-04 19:00:00+00:00
5769,198,0.0,2025-03-04 19:00:00+00:00
5770,20,0.0,2025-03-04 19:00:00+00:00
5771,248,0.0,2025-03-04 19:00:00+00:00
